# آٹو جن بنیادی نمونہ

اس کوڈ نمونے میں، آپ [AutoGen](https://aka.ms/ai-agents/autogen) AI فریم ورک استعمال کریں گے تاکہ ایک بنیادی ایجنٹ بنایا جا سکے۔

اس نمونے کا مقصد آپ کو وہ مراحل دکھانا ہے جو ہم بعد میں اضافی کوڈ نمونوں میں مختلف ایجنٹک پیٹرنز نافذ کرتے وقت استعمال کریں گے۔


## مطلوبہ پائتھن پیکجز درآمد کریں


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## کلائنٹ بنائیں

اس نمونے میں، ہم [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) کا استعمال کریں گے تاکہ LLM تک رسائی حاصل کی جا سکے۔

`model` کو `gpt-4o-mini` کے طور پر بیان کیا گیا ہے۔ مختلف نتائج دیکھنے کے لیے ماڈل کو GitHub Models مارکیٹ پلیس میں دستیاب کسی دوسرے ماڈل میں تبدیل کرنے کی کوشش کریں۔

ایک فوری ٹیسٹ کے طور پر، ہم ایک سادہ پرامپٹ چلائیں گے - `What is the capital of France`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## ایجنٹ کی تعریف

اب جب کہ ہم نے `client` کو ترتیب دے دیا ہے اور اس کے کام کرنے کی تصدیق کر لی ہے، آئیے ایک `AssistantAgent` بناتے ہیں۔ ہر ایجنٹ کو درج ذیل تفصیلات تفویض کی جا سکتی ہیں:  
**name** - ایک مختصر نام جو کثیر ایجنٹ فلو میں اس کا حوالہ دینے کے لیے مفید ہوگا۔  
**model_client** - وہ کلائنٹ جو آپ نے پچھلے مرحلے میں بنایا تھا۔  
**tools** - دستیاب ٹولز جنہیں ایجنٹ کسی کام کو مکمل کرنے کے لیے استعمال کر سکتا ہے۔  
**system_message** - وہ میٹا پرامپٹ جو LLM کے کام، رویے اور لہجے کو بیان کرتا ہے۔  

آپ سسٹم میسج کو تبدیل کر کے دیکھ سکتے ہیں کہ LLM کیسے ردعمل دیتا ہے۔ ہم `tools` کو سبق نمبر 4 میں تفصیل سے دیکھیں گے۔


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## ایجنٹ کو چلائیں

نیچے دی گئی فنکشن ایجنٹ کو چلائے گی۔ ہم `on_message` میتھڈ استعمال کرتے ہیں تاکہ ایجنٹ کی حالت کو نئے پیغام کے ساتھ اپ ڈیٹ کیا جا سکے۔

اس مثال میں، ہم صارف کی طرف سے ایک نئے پیغام کے ساتھ حالت کو اپ ڈیٹ کرتے ہیں، جو کہ ہے: `"مجھے ایک بہترین دھوپ والی چھٹی کا منصوبہ بنائیں"`۔

آپ پیغام کے مواد کو تبدیل کر سکتے ہیں تاکہ دیکھ سکیں کہ LLM مختلف طریقے سے کیسے جواب دیتا ہے۔


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**ڈس کلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا عدم درستگی ہو سکتی ہیں۔ اصل دستاویز، جو اس کی مقامی زبان میں ہے، کو مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے لیے ہم ذمہ دار نہیں ہیں۔
